# Midterm 1

## FINM 36700 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is `100` points.
* You have `125` minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.


Submission
* You will upload your solution to the `Midterm 1` assignment on Canvas, where you downloaded this. (Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,)
* and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_1_data.xlsx`

This file has sheets for...
* `stocks excess returns` - excess returns of the 14 biggest companies in the S&P.
* `proshares excess returns` - excess returns of ETFs and indexes from the Proshares case study.
* `fx carry excess returns` - excess returns from FX products.

Note the data is **monthly** for the first two sheets (stocks and proshares). Any annualizations for those two sheets should use `12` months in a year. Annualization for the third sheet (fx carry excess returns) is explained in section 4.

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 15     |
| 2       | 25     |
| 3       | 35     |
| 4       | 25     |

### Each numbered question is worth 5 points unless otherwise specified.

***

# 1. Short Answer

#### No Data Needed

These problems do not require any data file. Rather, analyze them conceptually. 

### 1.

#### (10pts)

In the mean-variance optimization of `homework 1`, suppose we found the mean excess return of TIPS is 4% annualized.

Explain--conceptually--how each of the following would have impacted the new (with TIPS) MV solution.
* TIPS is found to have correlation of 0% to `IEF` and 0% to SPY.
* TIPS is found to have correlation of 100% to `IEF`.

Would it be possible for TIPS to have been found to have 0% correlation to every other asset in `homework 1`? Explain.

TIPS already had quite a low correlation to our portfolio, and that is why it was so valuable, because it reduced the risk profile of our portfolio via diversification. If it would have had a 0% correlation, then that would have made it even more valuable. However, if it was highly corelated to IEF, than we would not have gained as much benefit from it. IEF is the ticker for 10 year treasury bonds, so including another highly correlated bond wouldn't have been very valuable unless TIPS had a large return. It is very unlikely TIPS would have a 0% correlation to the assets. Since there were already bonds in the portfolio, it was bond to have some sort of correlation with them.

### 2. 

Depending on the application, one may or may not choose to include an intercept term in a linear factor decomposition of an asset’s returns. In what circumstances would I prefer to include an intercept, and in what circumstances would I not?

An intercept makes sense when the intercept is investible. For example, if the intercept is the interest free rate, then we can absolutely use it. However, in the case when it is not investible, like in our example about replicating hedge fund returns where the intercept represented the alpha hedge funds generate, we cannot easily invest in it, and should not include it.

***

# 2. Portfolio Allocation

For this question you will only use data from the sheet `stocks excess returns`.

It contains excess returns for the 14 largest stocks in the S&P.

### 1.

Calculate the tangency portfolio from the start of the sample to December of 2018 (to 2018-12-31), which we call in-sample period. Use the following methods:
- Traditional tangency portfolio.
- Regularized tangency portfolio (divide by 2 every element outside of the diagonal in the covariance matrix prior to the calculation).

Return:
- The weights of each asset for the traditional tangency portfolio and the regularized tangency portfolio.
- The sum of absolute values of the weights for the traditional tangency portfolio and the regularized tangency portfolio:

$$
\sum_{i=1}^{n} |w_i|
$$


In [71]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")

In [72]:
#import data

#get all stock data
FILEIN = '../data/midterm_1_data.xlsx'
sheet_exrets = 'stocks excess returns'
retsx = pd.read_excel(FILEIN, sheet_name=sheet_exrets).set_index('date')

retsx.head()

,AAPL,AMZN,BRK-B,GOOGL,JNJ,JPM,LLY,META,MSFT,NVDA,TSLA,UNH,V,XOM
date,,,,,,,,,,,,,,
2012-06-30,0.010943,0.072609,0.050116,-0.001270,0.082263,0.077918,0.047953,0.050766,0.048052,0.111916,0.060768,0.052919,0.073267,0.088352
2012-07-31,0.045822,0.021677,0.018121,0.091196,0.024570,0.015958,0.026101,-0.301929,-0.036613,-0.020260,-0.123682,-0.126667,0.044002,0.014958
2012-08-31,0.093695,0.063985,-0.006075,0.082161,-0.017188,0.031486,0.031399,-0.168306,0.052556,0.036008,0.039935,0.062649,-0.004849,0.011533
2012-09-30,0.002878,0.024450,0.045849,0.101397,0.022030,0.090012,0.055749,0.199417,-0.034312,-0.049099,0.026730,0.024657,0.047099,0.047619
2012-10-31,-0.107527,-0.084192,-0.020903,-0.098271,0.027789,0.037321,0.025805,-0.025320,-0.040922,-0.101877,-0.039204,0.010720,0.033436,-0.002989


In [73]:
#splitting up in sample and out of sample data
retsx_IS = retsx.loc[:'2018']
retsx_OOS = retsx.loc['2019':]

In [74]:
#=using the function from the practice midterm for the tangency portfolio

def tangency_weights(returns, cov_mat = 1):
    
    if cov_mat ==1:
        cov_inv = np.linalg.inv((returns.cov()*12))
    else:
        cov = returns.cov()
        covmat_diag = np.diag(np.diag((cov)))
        covmat = cov_mat * cov + (1-cov_mat) * covmat_diag
        cov_inv = np.linalg.inv((covmat*12))  
        
    # ones = np.ones(returns.columns[1:].shape) 
    ones = np.ones((returns.shape[1], 1))

    # mu = returns.mean()*12
    mu = returns.mean().values.reshape(-1, 1)  # Convert mu to a column vector

    # print("ones shape:", ones.shape)
    # print("cov_inv shape:", cov_inv.shape)
    # print("mu shape:", mu.shape)
    
     # scaling = 1/(np.transpose(ones) @ cov_inv @ mu)
    scaling = 1 / (ones.T @ cov_inv @ mu)  # Transpose ones to make it a row vector
    
    tangent_return = scaling*(cov_inv @ mu)

    # tangency_wts = pd.DataFrame(index = returns.columns[1:], data = tangent_return, columns = ['Tangent Weights'] )
    tangency_wts = pd.DataFrame(index = returns.columns[:], data = tangent_return[:], columns = ['Tangent Weights'] )
        
    return tangency_wts

In [75]:
wts = pd.DataFrame(index = retsx_IS.columns, columns=['tangency','regularized'])
wts.loc[:,'tangency'] = tangency_weights(retsx_IS, cov_mat = 1).values
wts.loc[:,'regularized'] = tangency_weights(retsx_IS, cov_mat = (1/2)).values #regularizing by a factor of 1/2 as indicated in the question
wts

,tangency,regularized
AAPL,-0.127836,-0.014706
AMZN,-0.040576,0.03631
BRK-B,0.131333,0.109162
GOOGL,0.025968,0.050545
JNJ,0.130408,0.09919
JPM,-0.013929,0.053691
LLY,0.352670,0.214949
META,0.030541,0.034896
MSFT,0.137917,0.08997
NVDA,0.163501,0.071696


In [76]:
#chat GPT prompt: get absoulte value of each row in a data frame
wts_absolute = wts.abs()
sum_absolute = wts_absolute.sum()
sum_absolute

tangency       2.197687
regularized    1.342647
dtype: object

### 2.

Calculate the annualized summary statistics (mean, Sharpe, vol) of both portfolios in-sample.


In [77]:
#importing performance summary code from practice midterm

def performance_summary(return_data, annualization = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """

    if isinstance(return_data, pd.Series):
        return_data = return_data.to_frame('Returns') 
        
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [78]:
tan_statistics = performance_summary(retsx_IS @ wts['tangency'] , 12)
reg_statistics = performance_summary(retsx_IS @ wts['regularized'] , 12)

summary_df = pd.DataFrame(columns=['Mean', 'Volatility', 'Sharpe Ratio'])
selected_columns1 = tan_statistics[['Mean', 'Volatility', 'Sharpe Ratio']]
selected_columns1.index = ['Tangency Portfolio']

selected_columns2 = reg_statistics[['Mean', 'Volatility', 'Sharpe Ratio']]
selected_columns2.index = ['Regularized Portfolio']

summary_df = pd.concat([summary_df, selected_columns1])
summary_df = pd.concat([summary_df, selected_columns2])
summary_df


,Mean,Volatility,Sharpe Ratio
Tangency Portfolio,0.348256,0.134053,2.597896
Regularized Portfolio,0.276599,0.114184,2.422392


### 3.

Use the weights calculated in question (2.2) to produce portfolio returns out-of-sample for both the Traditional and Regularized portfolio (from January 2019 onwards). 

Report the **last 3 returns** of both portfolios in the out-of-sample (the traditional tangency portfolio and the regularized tangency portfolio).

In [79]:
#Chat GPT prompt: how to calulcate these weights vs these returns to calcualte a new weighted returns dataframe 
weighted_returns_tangency = retsx_OOS * wts['tangency'].values
weighted_returns_regularized = retsx_OOS * wts['regularized'].values

total_weighted_returns_tangency = weighted_returns_tangency.sum(axis=1)
total_weighted_returns_regularized = weighted_returns_regularized.sum(axis=1)

print("Last 3 returns:")

print("Tangency Portfolio")
print(total_weighted_returns_tangency.tail(3))

print("\n")

print("Regularized Portfolio")
print(total_weighted_returns_regularized.tail(3))

Last 3 returns:
Tangency Portfolio
date
2024-07-31   -0.016012
2024-08-31    0.101241
2024-09-30   -0.025162
dtype: float64


Regularized Portfolio
date
2024-07-31    0.003227
2024-08-31    0.064626
2024-09-30   -0.016477
dtype: object


### 4.

Report the annualized summary statistics (Mean, Vol and Sharpe) of both portfolios in the out-of-sample.

Note: you are using the weights optimized for the in-sample and generating statistics with the out-of-sample returns.

In [80]:
tan_statistics = performance_summary(retsx_OOS @ wts['tangency'] , 12)
reg_statistics = performance_summary(retsx_OOS @ wts['regularized'] , 12)

summary_df = pd.DataFrame(columns=['Mean', 'Volatility', 'Sharpe Ratio'])
selected_columns1 = tan_statistics[['Mean', 'Volatility', 'Sharpe Ratio']]
selected_columns1.index = ['Tangency Portfolio']

selected_columns2 = reg_statistics[['Mean', 'Volatility', 'Sharpe Ratio']]
selected_columns2.index = ['Regularized Portfolio']

summary_df = pd.concat([summary_df, selected_columns1])
summary_df = pd.concat([summary_df, selected_columns2])
summary_df

,Mean,Volatility,Sharpe Ratio
Tangency Portfolio,0.321558,0.222352,1.446167
Regularized Portfolio,0.268686,0.176334,1.523729


### 5.
Which portfolio has better adjusted by risk returns in the out-of-sample? Could there be a mathematical/optimization reason why one portfolio had better adjusted by risk performance? 

Relate your answer to your findings in question (2.2) (Sum of absolute weights in the traditional and regularized tangency portfolio.)

The regularized portfolio has better performance out of sample, which is what we saw in HW1. This is due to the way we "optimize" our portfolio. The matrix math ends up causing some weights to be too extreme without regularization, which is why the abs value of weights for the tangency portfolio are higher, causing the model to perform fine in sample but struggle out of sample.

***

# 3. Hedging and Replication

For this question you will only use data from the sheet `proshares returns`.

The following assets excess returns are available in this sheet:

- **HDG US Equity**: ProShares Hedge Replication ET
- **QAI US Equity**: NYLI Hedge Multi-Strategy Trac
- **SPY US Equity**: SPDR S&P 500 ETF Trust
- **EEM US Equity**: iShares MSCI Emerging Markets
- **EFA US Equity**: iShares MSCI EAFE ETF
- **EUO US Equity**: ProShares UltraShort Euro
- **IWM US Equity**: iShares Russell 2000 ETF
- **SPXU US Equity**: ProShares UltraPro Short S&P 5
- **UPRO US Equity**: ProShares UltraPro S&P 500

### 1. 

You work at a hedge fund.

Suppose the hedge fund is long $1 million of HDG and wants to hedge the position.

A junior analyst suggests that we can hedge our position by looking at some select ETFs, and then taking a position in the ETFs that will offset the risk of our HDG position.

They pick QAI, SPY, EEM, UPRO, SPXU, IWM, and EFA.

What dollar position would we be taking in each ETF to hedge your HDG position?

### 2.

#### (7pts)
What is the gross notional of the hedge?

What is the R-squared of the hedge?

What do these two statistics indicate about the practical use of this hedge?

### 3.

Suppose instead we don't want to hedge our position. We believe that the value of HDG can be *entirely* determined by some combination of the other ETFs. 

So, you propose the following model:

$$
HDG_t = \beta_1 QAI_t + \beta_2 SPY_t + \beta_3 EEM_t + \varepsilon_t
$$

We think any difference between the value of HDG and the value of the ETFs is a mispricing, and will revert to 0 in the future. We call such a strategy "trading the residuals".

Therefore, if $\varepsilon_t > 0$, we should be short HDG and long the basket, and if $\varepsilon_t < 0$, we should be long HDG and short the basket.

Now...
* Run the model specified above and report the $\beta$'s values.
* After, create the "basket" portfolio, using the $\beta$'s as weights (they do not need to add up to one). Report the final three values.

### 4.

#### (8pts)

Construct the strategy indicated by the approach in the previous problem.

For a given period $t$:
- if the $\varepsilon_t \leq 0$ (is negative or equal to 0), you should be long HDG 200% in HDG and short 100% in the basket portfolio **in period $t+1$**.
- if the $\varepsilon_t > 0$ (is positive), you should be long 200% in the basket portfolio and short 100% **in HDG in period $t+1$**.

Do not worry about the look forward bias: in this scenario, you should run the model only once with the entire dataset and define your $\varepsilon_t$ for any $t$ also considering the model that has acess to data in $t+1, t+2, ...$ to make the calculation.  

Report the annualized summary statistics of this strategy (Mean, Vol and Sharpe).

### 5.

On a different matter, we are now studying QAI and want to track (replicate) it using the other available ETFs.

Use an intercept and report:

- $\beta$ (and the sum of $\beta$'s absolute value).
- $\alpha$ and Information Ratio.
- $R^2$.
- Correlation matrix between the assets used to replicate QAI.

### 6.

Explain how good is your replication, pointing out at least one good or bad argument related to each of the statistics mentioned in the bullet points above (thus, you should have at least 4 arguments).

***

# 4. 

The data in sheet `fx carry excess returns` has **excess** daily returns for trading currencies.
* You **do NOT need** to know anything about FX, currency, or the underlying strategies.
* Rather, just take these return series as given.

For the problems below, we will **only use** the `JPY` series.

### 1.

Calculate the `1%` VaR as follows...

Empirical VaR:
* At every point in time, calculate the `1st` quantile of the returns up to that point.
* No need to scale the answers.
  
Report the VaR for the final date of the sample.

### 2.

Now calculate the `normal VaR` of `JPY` as follows,

$$\text{Normal VaR (1\%)} = -2.33\, \sigma_t$$

where $\sigma_t$ is estimated with
* rolling volatility.
* using a window of `233` days.
* without using a sample mean.

Report the VaR for the final `3 days` of the sample.

### 3.

Now calculate the `normal VaR` of `JPY` as follows,

$$\text{Normal VaR (1\%)} = -2.33\, \sigma_t$$

where $\sigma_t$ is estimated with
* EWMA volatility
* using $\lambda = 0.94$.
* without using a sample mean.

Report the VaR for the final `3 days` of the sample.

### 4.

Make a plot of the three timeseries of your VaR estimates.

Succinctly point out the pros / cons of these approaches.

### 5.

What statistic do we use to judge the performance of a VaR model?

Estimate and report this statistic across the VaR methods.

Which VaR model do you find is best?

***